In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from keras import optimizers
from keras.models import Sequential, Model 
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization, Input
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras.utils import plot_model

import cv2    

from sklearn.metrics import f1_score
from os import listdir
from os.path import isfile, join


aliens = ['../input/celeba-dataset/img_align_celeba/img_align_celeba/' + f for f in listdir('../input/celeba-dataset/img_align_celeba/img_align_celeba/') if isfile(join('../input/celeba-dataset/img_align_celeba/img_align_celeba/', f))]

del aliens[5000:202599]

In [ ]:
list_img = []
list_label = []
for i,img_info in enumerate(aliens):
    print (i, ' / ', len(aliens))
    img = load_img(img_info, target_size=(224,224,3))
    img_array = img_to_array(img)
    list_img.append(img_array)
    list_label.append(img_info)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
model = InceptionResNetV2(pooling='avg',include_top=False, weights='imagenet', input_shape=(224,224,3))
model.summary()

In [ ]:
import numpy as np
X = np.array(list_img)
X = preprocess_input(X)
list_finale = []
for index in range(0, len(list_img)):
    print (index, ' / ', len(list_img))
    ret_vec = model.predict(X[index].reshape(1,224,224,3))
    list_finale.append({'vec':ret_vec.reshape(-1),'original':list_img[index], 'path':list_label[index], 'index':index})

In [ ]:
from keras.preprocessing.image import array_to_img
to_find = 0
%matplotlib inline
import matplotlib.pyplot as plt


plt.imshow(array_to_img(list_finale[to_find]['original']))
plt.show()

In [ ]:
from scipy import spatial
cosine_list = []
for index_image,xt in enumerate(list_finale):
    print (index_image, ' / ', len(list_finale))
    result = 1 - spatial.distance.cosine(list_finale[to_find]['vec'].reshape(-1), xt['vec'])
    cosine_list.append(dict({'res':result, 'i':list_finale[index_image]['index']}))
from operator import itemgetter
cosine_list.sort(key=itemgetter('res'), reverse=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=10,figsize=(20, 4))
plt.gray()
for indice, row in enumerate(ax):
    print (cosine_list[indice]['i'])
    row.imshow(array_to_img(list_finale[cosine_list[indice]['i']]['original']))

plt.show()

In [ ]:
cosine_list